In [ ]:
!git clone https://github.com/SobhanRjz/DL_Detectron2.git
%cd DL_Detectron2

In [ ]:

# Look at training curves in tensorboard:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

from google.colab.patches import cv2_imshow



In [ ]:
import detectron2
print(f"Detectron2 version: {detectron2.__version__}")


In [3]:
import os
import sys
# Get the parent directory path
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)


import logging
from pathlib import Path
import torch
from Config.basic_config import BASE_PATH, DATA_PATH, OUTPUT_PATH, DEVICE
from Config.basic_config import detectron2_logger as logger
from Config.detectron2_config import DetectronConfig
from Config.dataset_config import DatasetConfig
from PreProcess.preprocess_COCO import COCOJsonProcessor
from Train import mainTrain
from detectron2.engine import launch


In [ ]:
DATA_PATH

In [ ]:
# Use pathlib for more robust path handling
json_paths = [
    Path(DATA_PATH) / split / "_annotations.coco.json"
    for split in ["train", "test", "valid"]
]

logger.info("Processing COCO JSON files...")
# coco_processor = COCOJsonProcessor([str(p) for p in json_paths])
# coco_processor.process_files()

logger.info("Registering datasets...")
DatasetConfig().register_datasets()


In [ ]:
from collections import Counter
from detectron2.data import DatasetCatalog, MetadataCatalog
import matplotlib.pyplot as plt
import matplotlib

# Load dataset

dataset_dicts = DatasetCatalog.get("my_dataset_train")
metadata = MetadataCatalog.get("my_dataset_train")



In [ ]:
def setup_training():
    """Configure training settings and get GPU count"""
    logger.info("Loading Detectron2 configuration...")

    
    detectron_config = DetectronConfig()
    cfg = detectron_config.get_cfg()

    num_gpus = torch.cuda.device_count()
    logger.info(f"Number of GPUs available: {num_gpus}")
    
    return cfg, num_gpus
# Setup training configuration
cfg, num_gpus = setup_training()

In [ ]:
logger.info("Launching training...")
trained_model = launch(
    mainTrain,
    num_gpus,
    num_machines=1,
    machine_rank=0,
    dist_url="auto", 
    args=(cfg, "default"),
)